In [1]:
import ipywidgets as widgets
from IPython.display import display

series_dropdown = widgets.Dropdown(
    options=["Cup", "Xfinity", "Trucks"],
    description="Series:"
)
display(series_dropdown)

prediction_type_dropdown = widgets.Dropdown(
    options=["Finishing Position", "Points Earned in Race"],
    description="Prediction Type:"
)
display(prediction_type_dropdown)

Dropdown(description='Series:', options=('Cup', 'Xfinity', 'Trucks'), value='Cup')

Dropdown(description='Prediction Type:', options=('Finishing Position', 'Points Earned in Race'), value='Finis…

In [2]:
# to be updtated with full race name dictionary soon for better UX
# Xfinity and Truck data not implemented yet, but structure is in place
if series_dropdown.value == "Cup":
    race_id_dropdown = widgets.Dropdown(
        options=[5575, 5577, 5578, 5567, 5579, 5581, 5582, 5580, 5584],
        description="Race ID:"
    )
    display(race_id_dropdown)

elif series_dropdown.value == "Xfinity":
    race_id_dropdown = widgets.Dropdown(
        options=[5534, 5535, 5537, 5538, 5539, 5537],
        description="Race ID:"
    )
    display(race_id_dropdown)

elif series_dropdown.value == "Trucks":
    race_id_dropdown = widgets.Dropdown(
        options=[5493, 5502, 5508, 5505, 5504],
        description="Race ID:"
    )
    display(race_id_dropdown)

else: 
    print("An error selecting series occurred.")

Dropdown(description='Race ID:', options=(5575, 5577, 5578, 5567, 5579, 5581, 5582, 5580, 5584), value=5575)

In [3]:
import pandas as pd
import sys
from pathlib import Path

# # make parent folder importable
parent_folder = Path.cwd().parent
sys.path.append(str(parent_folder))

# import config
from config import DATA_DIR

if prediction_type_dropdown.value == "Finishing Position":
    df = pd.read_csv("lr_analysis_ready_finishing.csv")
    target_col = "finishing_position"
elif prediction_type_dropdown.value == "Points Earned in Race":
    df = pd.read_csv("lr_analysis_ready_points.csv")
    target_col = "points_earned_rank"
else:
    print("An error selecting prediction type occurred.")

In [4]:
# race_id_to_check = 5580 
race_id_to_check = race_id_dropdown.value

race_df = df[df["race_id"] == race_id_to_check].copy()

# compute absolute error between predicted rank and actual finish
race_df["abs_error_weighted"] = (race_df["pred_rank_lr"] - race_df[target_col]).abs()

# compute summary metrics
mean_error = race_df["abs_error_weighted"].mean()
median_error = race_df["abs_error_weighted"].median()

print(f"Weighted model for race {race_id_to_check}:")
print(f"Mean absolute error: {mean_error:.2f}")
print(f"Median absolute error: {median_error:.2f}")

# Sort by predicted rank (best at the top)
race_df = race_df.sort_values("pred_rank_lr")

# Select useful columns to display
cols_to_show = [
    "race_id",
    "driver_fullname",
    # "team_name",
    # "weighted_score_lr_flipped",
    "pred_rank_lr",
    target_col  # actual for comparison
]

print(race_df[cols_to_show].head(42))
# race_df.to_csv("race_db.csv", index=False)

Weighted model for race 5584:
Mean absolute error: nan
Median absolute error: nan
      race_id      driver_fullname  pred_rank_lr  points_earned_rank
1316     5584        William Byron           1.0                 NaN
1173     5584        Chase Elliott           2.0                 NaN
1603     5584         Denny Hamlin           3.0                 NaN
1030     5584          Kyle Larson           4.0                 NaN
4724     5584          Ryan Blaney           5.0                 NaN
1782     5584        Chase Briscoe           6.0                 NaN
1925     5584     Christopher Bell           7.0                 NaN
4867     5584          Joey Logano           8.0                 NaN
5047     5584        Ross Chastain           9.0                 NaN
320      5584        Tyler Reddick          10.0                 NaN
4581     5584       Austin Cindric          11.0                 NaN
2032     5584             Ty Gibbs          12.0                 NaN
142      5584        

c:\Users\joeah\AppData\Local\Programs\Python\Python313\Lib\site-packages\numpy\lib\_nanfunctions_impl.py:1214: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis, out=out, keepdims=keepdims)


In [5]:
if target_col == "finishing_position" and series_dropdown.value == "Cup":    
    racing_insights_file = DATA_DIR / "10-20-racing-insights.csv"
    racing_insights = pd.read_csv(racing_insights_file)

    racing_insights["race_id"] = racing_insights["race_id"].astype(df["race_id"].dtype)
    racing_insights["driver_id"] = racing_insights["driver_id"].astype(df["driver_id"].dtype)

    df = pd.merge(df, racing_insights, on=["race_id", "driver_id"], how="outer", suffixes=("", "_racing_insights"))

    assert df.duplicated(subset=["race_id", "driver_id"]).sum() == 0

    df_2025 = df[df["race_season"] == 2025].copy()

    print(df_2025.columns.tolist())

else:
    df_2025 = df[df["race_season"] == 2025].copy()

In [6]:
df_2025["abs_error_my_model"] = (df_2025["pred_rank_lr"] - df_2025[target_col]).abs()
if target_col == "finishing_position" and series_dropdown.value == "Cup":
    df_2025["abs_error_ri"] = (df_2025["pred_finish_ri"] - df_2025["finishing_position"]).abs()

print("\nMean Absolute Error (2025):")
print("My model:", df_2025["abs_error_my_model"].mean().round(2))
if target_col == "finishing_position" and series_dropdown.value == "Cup":
    print("Racing Insights:", df_2025["abs_error_ri"].mean().round(2))


Mean Absolute Error (2025):
My model: 7.98


In [7]:
from scipy.stats import spearmanr
import numpy as np
import pandas as pd

def race_spearman_corrs(g):
    result = {}
    
    # Only calculate if we have at least 2 drivers
    if g[target_col].nunique() > 1:
        # Your model correlation
        if g["pred_rank_lr"].nunique() > 1:
            result["my_model_corr"] = spearmanr(g["pred_rank_lr"], g[target_col]).correlation
        else:
            result["my_model_corr"] = np.nan
        
        # Racing Insights correlation
        if target_col == "finishing_position" and series_dropdown.value == "Cup":
            if g["pred_finish_ri"].nunique() > 1:
                result["ri_corr"] = spearmanr(g["pred_finish_ri"], g["finishing_position"]).correlation
            else:
                result["pred_finish_ri"] = np.nan
    
    return pd.Series(result)

race_corrs_2025 = (
    df_2025.dropna(subset=[target_col])
    .groupby("race_id")
    .apply(race_spearman_corrs)
)

C:\Users\joeah\AppData\Local\Temp\ipykernel_18204\1946305589.py:28: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(race_spearman_corrs)


In [8]:
print("Race-by-race Spearman Correlations (2025):")
print(race_corrs_2025.round(3))

print("\nAverage correlations across 2025:")
print("My model:", race_corrs_2025["my_model_corr"].mean().round(3))
if target_col == "finishing_position" and series_dropdown.value == "Cup":
    print("Racing Insights:", race_corrs_2025["ri_corr"].mean().round(3))

Race-by-race Spearman Correlations (2025):
         my_model_corr
race_id               
5546             0.587
5547             0.424
5548             0.441
5549             0.511
5550             0.798
5551             0.491
5552             0.489
5553             0.691
5554             0.258
5555             0.274
5556             0.728
5557             0.536
5558             0.569
5563             0.444
5564             0.748
5565             0.502
5566             0.687
5567             0.686
5568             0.753
5569             0.573
5570             0.638
5571             0.534
5572             0.517
5573             0.526
5574             0.266
5575             0.606
5576             0.218
5577             0.673
5578             0.444
5579             0.546
5580             0.379
5581             0.777
5582             0.632
5583             0.644

Average correlations across 2025:
My model: 0.547
